In [59]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [60]:
data= pd.read_csv("C:/Users/Kareem/Desktop/annclassification/annclassification/Churn_Modelling.csv")

In [61]:
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [62]:
print(type(data["Geography"]))

print(type(data[["Geography"]]))
encoder=OneHotEncoder()
geography_encoded=encoder.fit_transform(data[["Geography"]]).toarray()

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [63]:
geography_encoded_pd=pd.DataFrame(geography_encoded,columns=encoder.get_feature_names_out(["Geography"]))

In [64]:
X=pd.concat([data.drop(["RowNumber","Geography","CustomerId","Surname"],axis=1),geography_encoded_pd],axis=1)

In [65]:
y=X["Exited"]
X=X.drop(["Exited"],axis=1)
encoder=LabelEncoder()
X["Gender"]=encoder.fit_transform(X['Gender'])

X.head(5)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


In [66]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,shuffle=True,random_state=42)

In [67]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [ ]:
log_dir="D:Nlp_course/log2/fit"
tensorBoard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
EarlyStopping_callback=EarlyStopping(monitor='accuracy'
,patience=10,restore_best_weights=True)

In [87]:

def model(neurons=32,layers=2):
    model= Sequential()
    model.add(Dense(neurons,activation="relu",input_shape=(X.shape[1],)))
    for _ in range(layers-1):
      model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss="binary_crossentropy",metrics=['accuracy'],optimizer='Adam')
    return model

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
kera=KerasClassifier(model=model,epochs=100,batch_size=10,callbacks=[EarlyStopping_callback, tensorBoard_callback])

param_grid={
  'model__neurons': [16, 32, 64, 128],
    'model__layers': [1, 2],
   'epochs':[30,50]
}    

      
grid=GridSearchCV(estimator=kera,param_grid=param_grid,cv=3)
grid.fit(X_train,y_train)

print(grid.best_score_,grid.best_params_)

Epoch 1/30


d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6692 - loss: 818.3029 
Epoch 2/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6797 - loss: 46.9121
Epoch 3/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6747 - loss: 48.8901
Epoch 4/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6835 - loss: 47.8095
Epoch 5/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6848 - loss: 44.3530
Epoch 6/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6794 - loss: 43.1971
Epoch 7/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6942 - loss: 46.1200
Epoch 8/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6803 - loss: 47.7819
Epoch 9/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6859 - loss: 37.2396
Epoch 10/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6880 - loss: 36.3411
Epoch 11/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6820 - loss: 42.6225
Epoch 12/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/st

d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6940 - loss: 836.8267 
Epoch 2/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6818 - loss: 164.8924
Epoch 3/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6745 - loss: 101.9271
Epoch 4/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6758 - loss: 65.7507
Epoch 5/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6825 - loss: 54.9868
Epoch 6/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6824 - loss: 44.9825
Epoch 7/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6831 - loss: 50.1441
Epoch 8/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6840 - loss: 44.9047
Epoch 9/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6857 - loss: 39.8957
Epoch 10/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6859 - loss: 61.4929
Epoch 11/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6876 - loss: 52.0868
Epoch 12/30
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/

...

534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6878 - loss: 486.0305
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6754 - loss: 278.1774
Epoch 3/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6773 - loss: 207.7318
Epoch 4/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6765 - loss: 163.7180
Epoch 5/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6840 - loss: 104.5305
Epoch 6/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6745 - loss: 89.5084
Epoch 7/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6767 - loss: 78.3895
Epoch 8/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6822 - loss: 54.4513
Epoch 9/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6792 - loss: 49.3933
Epoch 10/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6769 - loss: 36.9245
Epoch 11/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6717 - loss: 32.2461
Epoch 12/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms

d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6745 - loss: 491.5975
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6742 - loss: 293.2981
Epoch 3/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6820 - loss: 263.0872
Epoch 4/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6764 - loss: 151.7502
Epoch 5/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6804 - loss: 139.3745
Epoch 6/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6811 - loss: 97.5748 
Epoch 7/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6777 - loss: 75.1034
Epoch 8/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6729 - loss: 72.7808
Epoch 9/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6807 - loss: 52.5410
Epoch 10/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6783 - loss: 51.7351
Epoch 11/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6755 - loss: 34.0883
Epoch 12/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 3m

d:\NLP_COURSE\nlp\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6756 - loss: 458.0147
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6766 - loss: 252.8819
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6810 - loss: 184.8583
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6791 - loss: 106.0557
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6850 - loss: 87.7397
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6823 - loss: 68.0596
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6804 - loss: 55.2180
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6809 - loss: 29.0227
Epoch 9/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6812 - loss: 28.2133
Epoch 10/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6808 - loss: 18.3633
Epoch 11/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6755 - loss: 14.4204
Epoch 12/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/

In [88]:
best_keras_model = grid.best_estimator_.model_
best_keras_model.save('best_model.keras')

In [89]:
%load_ext tensorboard 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [90]:
%tensorboard --logdir D:Nlp_course/log2/fit



Reusing TensorBoard on port 6006 (pid 3856), started 1:52:25 ago. (Use '!kill 3856' to kill it.)